# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

In [11]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from scipy.linalg import solve_discrete_are

# Load labeled matrices
A = pd.read_csv("model-A.tsv", sep="\t", index_col=0)
B = pd.read_csv("model-B.tsv", sep="\t", index_col=0)
Q = pd.read_csv("cost-Q.tsv", sep="\t", index_col=0)
R = pd.read_csv("cost-R.tsv", sep="\t", index_col=0)

A_mat = A.values.astype(float)
B_mat = B.values.astype(float)
Q_mat = Q.values.astype(float)
R_mat = R.values.astype(float)

# Solve the discrete-time algebraic Riccati equation (DARE)
P = solve_discrete_are(A_mat, B_mat, Q_mat, R_mat)

# Infinite-horizon discrete LQR gain:
# K = (R + B^T P B)^{-1} (B^T P A)
K_mat = np.linalg.inv(R_mat + B_mat.T @ P @ B_mat) @ (B_mat.T @ P @ A_mat)


Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

In [ ]:
# YOUR CHANGES HERE
# Keep labels for clarity and save
K = pd.DataFrame(K_mat, index=R.index, columns=A.columns)
K.to_csv("control-K.tsv", sep="\t", index=True)

print("K (u row over [x, v, a] columns):")
display(K)

K (u row over [x, v, a] columns):


,x,v,a
index,,,
u,0.33446,1.304454,1.858131


Submit "control-K-intended.tsv" in Gradescope.

## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [13]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

# Load K from Part 1 (expects file created previously)
K = pd.read_csv("control-K.tsv", sep="\t", index_col=0).iloc[0]
# Ensure we have coefficients aligned to [x, v, a]
kx, kv, ka = float(K["x"]), float(K["v"]), float(K["a"])

# Load the log of states/actions
log = pd.read_csv("qc-log.tsv", sep="\t")

# u_check = -K x_t  (discrete LQR uses u = -K x)
u_check = -(kx*log["x"] + kv*log["v"] + ka*log["a"])

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [14]:
# YOUR CHANGES HERE

# Save required file
out = pd.DataFrame({"index": log["index"], "u_check": u_check})
out.to_csv("qc-check.tsv", sep="\t", index=False)

# Quick sanity peek
display(out.head())
print("Saved qc-check.tsv")

,index,u_check
0,0,1.672299
1,1,-1.152095
2,2,-1.235183
3,3,-0.288504
4,4,0.369202


Saved qc-check.tsv


Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [15]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

# Load flight log
log = pd.read_csv("qc-log.tsv", sep="\t")

# Design matrix and target (no intercept).  u ≈ X @ beta  where X=[x,v,a]
X = log[["x","v","a"]].to_numpy()
y = log["u"].to_numpy()

# Least squares fit
beta, *_ = np.linalg.lstsq(X, y, rcond=None)

# For LQR form u = -K x  →  K_actual = -beta^T
K_actual = pd.DataFrame([-beta],
                        index=["u"],
                        columns=["x","v","a"])

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [16]:
# YOUR CHANGES HERE

# Save in required formats
K_actual.to_csv("control-K-actual.tsv", sep="\t")
K_actual.to_csv("actual-K.tsv", sep="\t")

# Quick sanity check
y_hat = X @ beta
ss_res = np.sum((y - y_hat)**2)
ss_tot = np.sum((y - y.mean())**2)
r2 = 1 - ss_res/ss_tot if ss_tot > 0 else np.nan

print("K_actual (row u over [x, v, a]):")
display(K_actual)
print(f"R^2 for u fit: {r2:.6f}")
print("Saved: control-K-actual.tsv and actual-K.tsv")

K_actual (row u over [x, v, a]):


,x,v,a
u,0.340438,1.30012,1.950117


R^2 for u fit: 1.000000
Saved: control-K-actual.tsv and actual-K.tsv


Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [17]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

# Load the noisy-action training log
df = pd.read_csv("qc-train.tsv", sep="\t").sort_values("index").reset_index(drop=True)

# Build (x_t, u_t) → x_{t+1}
X_t  = df[["x","v","a"]].to_numpy()[:-1, :]     # state at time t
u_t  = df[["u"]].to_numpy()[:-1, :]             # action column vector
Y_tp = df[["x","v","a"]].to_numpy()[1:, :]      # state at time t+1

# Design matrix with action: [x_t, v_t, a_t, u_t]
X_design = np.hstack([X_t, u_t])                # shape (n-1, 4)

# Solve Y_{t+1} ≈ X_design @ Θ  (least squares)
Theta, *_ = np.linalg.lstsq(X_design, Y_tp, rcond=None)
# Θ rows: [x, v, a, u]; columns: next-state components [x', v', a']

# Map Θ → A (3x3), B (3x1)
A_new = Theta[0:3, :].T                         # (3x3)
B_new = Theta[3, :].reshape(3, 1)               # (3x1)

# Labeled DataFrames for clarity
A_df = pd.DataFrame(A_new, index=["x","v","a"], columns=["x","v","a"])
B_df = pd.DataFrame(B_new, index=["x","v","a"], columns=["u"])

# Diagnostics
Y_hat = X_design @ Theta
rmse = np.sqrt(np.mean((Y_tp - Y_hat)**2, axis=0))

print("Estimated A:")
display(A_df)
print("\nEstimated B:")
display(B_df)
print("\nRMSE per component [x', v', a']:", rmse)



Estimated A:


,x,v,a
x,1.000000e+00,1.100000e+00,2.886580e-15
v,-1.281955e-16,9.000000e-01,9.500000e-01
a,-9.313780e-17,5.551115e-16,1.100000e+00



Estimated B:


,u
x,-1.110223e-16
v,-1.000000e-02
a,9.000000e-01



RMSE per component [x', v', a']: [9.48993751e-16 3.37486040e-16 2.55714247e-16]


Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [18]:
# YOUR CHANGES HERE

# If A_df/B_df aren’t in memory (rerun-safe), recompute quickly
try:
    A_df, B_df
except NameError:
    import pandas as pd, numpy as np
    df = pd.read_csv("qc-train.tsv", sep="\t").sort_values("index").reset_index(drop=True)
    X_t  = df[["x","v","a"]].to_numpy()[:-1, :]
    u_t  = df[["u"]].to_numpy()[:-1, :]
    Y_tp = df[["x","v","a"]].to_numpy()[1:, :]
    X_design = np.hstack([X_t, u_t])
    Theta, *_ = np.linalg.lstsq(X_design, Y_tp, rcond=None)
    A_df = pd.DataFrame(Theta[0:3, :].T, index=["x","v","a"], columns=["x","v","a"])
    B_df = pd.DataFrame(Theta[3, :].reshape(3,1), index=["x","v","a"], columns=["u"])

A_df.to_csv("model-A-new.tsv", sep="\t")
B_df.to_csv("model-B-new.tsv", sep="\t")

print("Saved model-A-new.tsv and model-B-new.tsv")


Saved model-A-new.tsv and model-B-new.tsv


Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [19]:
ack_text = """Acknowledgments

I used ChatGPT as a study partner to clarify  and check errors, and simplify code while completing this project. 


For transparency, I am including links to the transcripts of these interactions:

1. ChatGPT guidance: [https://chatgpt.com/share/68d20a8d-0870-8007-bb79-080efab54c33]
These resources document the support I received and ensure compliance with the class generative AI policy.
"""

with open("acknowledgments.txt","w") as f:
    f.write(ack_text)
